In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product
from scipy.spatial import ConvexHull, Delaunay


from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.inks import Neugebauer, InkGamut, CellNeugebauer, Pigment, InkLibrary
from chromalab.spectra import Spectra, Illuminant
from chromalab.maxbasis import MaxBasis
from chromalab.cubemap import CubeMap
from chromalab.printer import Printer

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [3]:
wavelengths = np.arange(390, 700, 10)
d65 = Illuminant.get("E")

trichromat = Observer.trichromat(wavelengths=wavelengths, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
LMS_to_RGB = maxbasis_tri.get_cone_to_maxbasis_transform()[::-1]

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis = MaxBasis(tetrachromat, verbose=True)
tetra_maxbasis = maxbasis.get_max_basis_observer()

100%|██████████| 28/28 [00:00<00:00, 50.66it/s]


In [4]:
# Max Contrast Hypercube Point Cloud
# stepsize = 0.05
# values = np.arange(0, 1 + stepsize, stepsize)/ np.linalg.norm([1, 1, 1, 1])
# total_combinations = len(values) ** 4
# LMSQ_gamut = product(values, repeat=4)
# enumerate_pc = np.array(list(LMSQ_gamut))

# # Convert to RGB
# enumerate_rgbs = np.clip((LMS_to_RGB@(enumerate_pc[:, [0, 1, 3]].T)).T, 0, 1)

# hull = Delaunay(point_cloud)
# def in_hull(p, hull):
#     """
#     Test if points in `p` are in `hull`

#     `p` should be a `NxK` coordinates of `N` points in `K` dimensions
#     `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
#     coordinates of `M` points in `K`dimensions for which Delaunay triangulation
#     will be computed
#     """
#     from scipy.spatial import Delaunay
#     if not isinstance(hull,Delaunay):
#         hull = Delaunay(hull)

#     return hull.find_simplex(p)>=0
# results = in_hull(enumerate_pc, hull)

In [5]:
point_cloud, rgbs = tetrachromat.get_enumeration_of_solid(0.05)

100%|██████████| 8990/8990 [00:21<00:00, 417.24it/s]


In [6]:
# fig = plt.figure(figsize=(6, 6))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(point_cloud[:, 1], point_cloud[:, 2], point_cloud[:, 3], s=70, c='blue', alpha=0.8, label='Points')
# ax.legend()
# plt.show()

In [7]:
import tetrapolyscope as ps

In [8]:
ps.set_background_color(np.array([0.0, 0.0, 0.0]))

In [9]:
cubemap = CubeMap(point_cloud, maxbasis, rgbs=rgbs, verbose=True)
# cubemap.display_cubemap_Q_orientation(0.8, 0.4, 5) # lum val, sat val, grid
cubemap.display_cubemap_in_ps(0.8, 0.4, 5)

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1
[0.23938265716671792, 0.53198425902075563]
[ 0.71663652  0.68487273  0.57228316  0.40022173  0.34265367  0.7600696
  0.70716061  0.59086987  0.58960188  0.4323889   0.91776774  0.89176607
  0.76345009  0.67185917  0.36846055  0.97492872  0.83879375  0.79100858
  0.58711619  0.52615868  0.95766829  0.91394621  0.75700497  0.76135975
  0.61060055  0.6961953   0.55716732  0.56762818  0.53652971  0.55134578
  0.52038928  0.55096165  0.46145044  0.38906878  0.37278638  0.41966282
  0.39868704  0.35493809  0.22921375  0.22921375  0.35524756  0.26730214
  0.21653873  0.1844848   0.15905624  0.26789454  0.18083982  0.12136465
  0.15194012  0.09233117  0.15687053  0.18083982  0.07570548  0.13718008
  0.08134322  0.33533274  0.06925275  0.19082153  0.13718008  0.
  0.34947665  0.21551377  0.31779155  0.19849661  0.14426477  0.43691668
  0.33058614  0.33058614  0.34558011  0.2965599   0.52839537  0.52839537
  0.43948897 

In [10]:
ps.rasterize_tetra('out.png')

: 